In [1]:
#getting our own spotify data

In [3]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [6]:
#creating our credentials



SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.